In [58]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from numpy import random
import os

In [64]:
def tokenize(s,max_length=1014): 
    A = []
    m = min(max_length,len(s))
    for i in range(m):
        c = s[i]
        A.append(ord(c))
    return np.array(A)
        
def dataGenerator(train_split=0.8,binary=False): 
    reviews = []
    ratings = []
    
    for filename in ['pos', 'neg']:
        cwd=os.getcwd()
        print(cwd)
        files = [f for f in listdir(join(cwd, 'train', filename)) if isfile(join(filename, f))]
        print(len(files))
        for f in files: 
            name = f.split('.')[0]
            name = name.split('_')
            id = name[0]
            rating = name[1]
            if(binary): 
                if(filename=='pos'): 
                    rating=1
                else: 
                    rating=0
            ratings.append(rating)
            path = filename+'/'+f
            with open(path) as myfile:
                data=myfile.read()
                data=tokenize(data)
                reviews.append(data)
                
    reviews = np.array(reviews)
    ratings = np.array(ratings,dtype=float)
    #rewiews, ratings
    
    #random split 0.8 / 0.2
    reviews_train, reviews_val, ratings_train, ratings_val =  train_test_split(reviews, ratings, test_size=1-train_split)
    return reviews_train, ratings_train, reviews_val, ratings_val

def batchify(review, rating, batch_size=32):   
    batches1 = []
    batches2 = []
    dataset_size = len(review)
    start = -1 * batch_size
    order = np.arange(dataset_size)
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch1 = [review[index] for index in batch_indices]
        batch2 = [rating[index] for index in batch_indices]
        batches1.append(batch1)
        batches2.append(batch2)
        
    return np.array(batches1), np.array(batches2)

In [65]:
reviews_train, ratings_train, reviews_val, ratings_val = dataGenerator()

/Users/saad/Downloads/NLP/NLP_project/characterLevelCNN
0
/Users/saad/Downloads/NLP/NLP_project/characterLevelCNN
0


In [22]:
class Char_CNN_Small(nn.Module):
    
    def __init__(self,
                 fully_layers,
                 l0,
                 alphabet_size,
                 nb_classes,
                 batch_size,
                 ):
        super(Char_CNN_Small,self).__init__()
        
        self.conv_layers = [
                    [256, 7, 3],
                    [256, 7, 3],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, 3]
                    ]
        self.fully_layers = fully_layers
        self.batch_size = batch_size
        self.nb_classes = nb_classes
        
        self.convs = []
        self.linear = []
        self.max_pool = nn.MaxPool1d(3)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout()
        
        in_feat = alphabet_size
        for out_feat, kernel_size, max_pool in conv_layers:
            conv = nn.Conv1d(in_feat, out_feat, kernel_size)
            self.convs.append(conv)
            in_feat = out_feat
        
        l6 = int((l0 - 96)/27)
        in_feat = l6*out_feat
        
        for out_feat in fully_layers:
            self.linear.append(nn.Linear(in_feat, out_feat))
            in_feat = out_feat
        
        self.classifier = nn.Linear(in_feat, nb_classes)
        
        if self.nb_classes == 2:
            self.class_non_lin = nn.Sigmoid()
        else:
            self.class_non_lin = nn.Softmax()
        

    def forward(self, x):
        out = x 
        print(out.size())
        for conv in self.convs[:2]:
            out = conv(out)
            out = self.relu(out)
            out = self.max_pool(out)
            print(out.size())
            
        for conv in self.convs[2:5]:
            out = conv(out)
            out = self.relu(out)
            print(out.size())
            
            
        out = self.convs[5](out)
        out = self.relu(out)
        out = self.max_pool(out)
        print(out.size())
        
        out = out.view(batch_size, -1)
        print(out.size())
        
        for lin in self.linear:
            out = lin(out)
            out = self.relu(out)
            out = self.dropout(out)
            print(out.size())

            
        out = self.classifier(out)
        out = self.class_non_lin(out)
        
        return out
    
    def init_weights(self):
        for conv in self.convs:
            nn.init.normal(conv.weight, mean=0, std=0.02)

In [23]:
def evaluate(model, data_iter):
    model.eval()
    correct = 0
    total = 0
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        #vectors = Variable(torch.stack(vectors).squeeze())
        #labels = torch.stack(labels).squeeze()
        
        output, hidden = model(vectors, hidden)
        
        predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
      
    return correct / float(total)

In [35]:
def training_loop(batch_size, num_epochs, model, loss_, optim, training_iter, eval_iter, train_eval_iter):
    step = 0
    epoch = 0

    while epoch <= num_epochs:
        model.train()
        for vectors, labels in training_iter:
            vectors, labels = next(training_iter) 
           # vectors = Variable(torch.stack(vectors).squeeze()) # batch_size, seq_len
           # labels = Variable(torch.stack(labels).squeeze())

            model.zero_grad()

            output = model(vectors, hidden, c_t)

            lossy = loss_(output, labels)
            lossy.backward()
            torch.nn.utils.clip_grad_norm(model.parameters(), 5.0)
            optim.step()
        
        if epoch % 1 == 0:
            print("Epoch %i; Step %i; Loss %f; Train acc: %f; Eval acc %f" 
                  %(epoch, step, lossy.data[0],\
                    evaluate(model, train_eval_iter, lstm),\
                    evaluate(model, eval_iter, lstm)))
        epoch += 1
        next(training_iter)

In [49]:
## Data Loading 
reviews_train, ratings_train, reviews_val, ratings_val = dataGenerator()


In [44]:
# Hyper Parameters 
conv_layers = [
                    [256, 7, 3],
                    [256, 7, 3],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, 3]
                    ]

fully_layers = [1024, 1024]
l0 = 1014
alphabet_size = 69
nb_classes = 4
batch_size = 256

learning_rate = 0.2
num_epochs = 100


# Build, initialize model
model = Char_CNN_Small(fully_layers, l0, alphabet_size, nb_classes, batch_size)
model.init_weights()

# Loss and Optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
training_iter = batchify(reviews_train, ratings_train, batch_size)
train_eval_iter = batchify(reviews_train[:500], ratings_train[:500],batch_size)
eval_iter = batchify(reviews_val, ratings_val, batch_size)

training_loop(batch_size, num_epochs, model, loss, optimizer, training_iter, eval_iter, train_eval_iter)

ValueError: not enough values to unpack (expected 2, got 0)

In [33]:
total_batches = int(len(training_iter) / batch_size)

In [38]:
for vectors, labels in training_iter:
    print(vectors.shape)

ValueError: not enough values to unpack (expected 2, got 0)

In [52]:
102*1024

104448

In [42]:
reviews_train.shape

(0,)